In [1]:
# stdlib
import os

# syft absolute
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [3]:
# node = sy.orchestra.launch(
#     name="test-domain-helm2",
#     dev_mode=True,
#     reset=True,
#     n_consumers=4,
#     create_producer=True,
# )
# client = node.login(email="info@openmined.org", password="changethis")

client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

Logged into <test: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
# [x.wait() for x in job.subjobs]

In [5]:
# stdlib

In [6]:
# import random
# import string

# def generate_random_line(length):
#     return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

# def generate_large_file(file_path, num_lines, line_length):
#     with open(file_path, 'w') as file:
#         for _ in range(num_lines):
#             line = generate_random_line(line_length)
#             file.write(line + '\n')

# file_path = "large_file.txt"
# num_lines = 34359712  # Adjust the number of lines as needed
# line_length = 1000  # Adjust the line length as needed

# generate_large_file(file_path, num_lines, line_length)

In [7]:
# with open("somefile16gb.txt", 'rb') as input_file, open("somefile32gb.txt", 'ab') as output_file:
#     while True:
#         chunk = input_file.read(50 -random.randint(1, 10))
#         if not chunk:
#             break
#         output_file.write(chunk)
# #     while True:
# #         chunk = input_file.read(50 -random.randint(1, 10))
# #         if not chunk:
# #             break
# #         output_file.write(chunk)

In [8]:
# lines_per_gb = int(2**30 / 1000)
# gbs=32
# with open(f'somefilerandom{gbs}gb.txt', 'wb') as f:
#     for x in range(lines_per_gb*gbs):
#         f.write(str.encode(''.join(random.choices(string.ascii_uppercase + string.digits, k=1000))) + b'\n')

In [9]:
# stdlib
from pathlib import Path

# syft absolute
from syft.client.client import SyftClient
from syft.store.blob_storage import BlobDeposit
from syft.types.blob_storage import CreateBlobStorageEntry

In [10]:
def allocate_file(client: SyftClient, path: Path) -> BlobDeposit:
    create_blob_storage_entry = CreateBlobStorageEntry.from_path(path)
    return client.api.services.blob_storage.allocate(create_blob_storage_entry)

In [11]:
file = "somefile32gb.txt"

In [12]:
dep = allocate_file(client, Path(file))

In [13]:
dep

```python
class SeaweedFSBlobDeposit:
  id: str = 741c4e3f5cfd4705ba3d12fc076b19dd

```

In [14]:
# from syft.client.api import APIRegistry

# api = APIRegistry.api_for(
#     node_uid=dep.syft_node_location,
#     user_verify_key=dep.syft_client_verify_key,
# )

# url = dep.urls[0]

# if api is not None:
#     blob_url = api.connection.to_blob_route(
#         url.url_path, host=url.host_or_ip
#     )
# else:
#     blob_url = url

# url = str(blob_url)

In [15]:
# import requests

# from syft import SyftError

# from io import BytesIO

# from typing import Generator

# def _byte_chunks(bytes: BytesIO, size: int) -> Generator[bytes, None, None]:
#     while True:
#         try:
#             yield bytes.read(size)
#         except BlockingIOError:
#             return

# DEFAULT_CHUNK_SIZE = (1024**3 // 10)  # 100MB
# DEFAULT_TIMEOUT = 5000  # in seconds

# print(len(dep.urls))

In [16]:
# third party

In [17]:
# byte_chunk[16384: 16384+10000]

In [18]:
# # relative
# with open(file, "rb") as data:
#     from syft.client.api import APIRegistry

#     api = APIRegistry.api_for(
#         node_uid=dep.syft_node_location,
#         user_verify_key=dep.syft_client_verify_key,
#     )

#     etags = []

#     try:
#         no_lines = 0
#         for part_no, (byte_chunk, url) in tqdm(enumerate(
#             zip(_byte_chunks(data, DEFAULT_CHUNK_SIZE), dep.urls),
#             start=1,
#         )):
#             no_lines += byte_chunk.count(b"\n")
#             if api is not None:
#                 blob_url = api.connection.to_blob_route(
#                     url.url_path, host=url.host_or_ip
#                 )
#             else:
#                 blob_url = url

#             def data_generator(bytes_, chunk_size=819200):
#                 n=0
#                 while n*chunk_size <= len(bytes_):
#                     chunk = bytes_[n*chunk_size:(n+1)*chunk_size]
#                     n += 1
#                     yield chunk

#             response = requests.put(
#                 url=str(blob_url), data=data_generator(byte_chunk), timeout=DEFAULT_TIMEOUT, stream=True
#             )
#             response.raise_for_status()
#             etag = response.headers["ETag"]
#             etags.append({"ETag": etag, "PartNumber": part_no})
#     except requests.RequestException as e:
#         print(e)

In [19]:
# len([x for x in data_generator(byte_chunk)])

In [20]:
# (1024 ** 3) // 10

In [21]:
# api.services.blob_storage.mark_write_complete(etags=etags, uid=dep.blob_storage_entry_id, no_lines=no_lines)

In [22]:
# mark_write_complete_method = from_api_or_context(
#     func_or_path="blob_storage.mark_write_complete",
#     syft_node_location=self.syft_node_location,
#     syft_client_verify_key=self.syft_client_verify_key,
# )
# return mark_write_complete_method(
#     etags=etags, uid=self.blob_storage_entry_id, no_lines=no_lines
# )

In [23]:
# with open("somefile.txt", "rb") as f:
#     dep.write(f)

In [24]:
# syft absolute
from syft.types.blob_storage import BlobFile

In [25]:
x = BlobFile.upload_from_path("somefile32gb.txt", client)

Uploading
7
4913438816
34394071712
1


Uploading file part 1/7: 100%|██████████████████████████████████████████████████████████████████████████| 5998/5998 [01:47<00:00, 55.76it/s]


2


Uploading file part 2/7: 100%|██████████████████████████████████████████████████████████████████████████| 5998/5998 [01:39<00:00, 60.00it/s]


3


...

Uploading file part 5/7: 100%|██████████████████████████████████████████████████████████████████████████| 5998/5998 [01:45<00:00, 56.86it/s]


6


Uploading file part 6/7: 100%|██████████████████████████████████████████████████████████████████████████| 5998/5998 [01:52<00:00, 53.14it/s]


7


Uploading file part 7/7: 100%|██████████████████████████████████████████████████████████████████████████| 5998/5998 [01:42<00:00, 58.46it/s]


In [26]:
for i, y in enumerate(x.iter_lines()):
    if i > 3:
        break
    print(y)

b'YEXFIA79OPO0ODMF7Z6SCGCYYHF8EIZ6NWP13WYVFPO7IC2W4CK6EDDCA7V1ITBU2F6IDLLAKX6RP39CDJF7THP30416ACJEGIXMFCGL25IRP6OCH6UEKGNYTL3RVSBI75T0DC2YIP98P0QBAYNEGBDFHENES15PBKOENVNBNTKW4MSGD3V74MDX0QIT5GM386O9QBPBD5NOFVR1GCSWT45A35O2ODJHFQ6HSLE0W5PNH2KWNGYP0OTOGG9DQ9JRHCDIORLLV1TJRDPXRITYKU7URHY0VVQ7G03VZ430A3CBEME6HBBAA0T6EYIM2M0ARIQYMAEAO3YLKOOSG868Z5BQXNY1FWMPQQY6NC942LKPZP9C8VK6WR6PLY926F5VV2G5E304T0RRB6D8I67G8D35SP60PJ3JUOIPW7NKMP99XMG2Z0UTT4SKWM1JW8H1DFOKOJD2OHE157V5HMFGU21VQRHN34M57M6EN7WGU27UKHENPC6DEV36ARCGGUX0A85JJTK0PADOOTLW1M5GDIX02O681LR2PY6ONJI71679VTRKVNCPLCSFQZV4XMR6NYFPDA3TAZCMUUH3HKJY8E4XY6D1FSKEJDC9P4UW05NJTAGAFQNG770F6IRYGKV29TLM1VJUNUYKY6PLYR4CSJ68HC8K5F1ZTDX9UEHRHZ2DAJ8Q8BJ5E2FNQIGZ8WJE4U2PG1BWDIXOXW8QWPQTAVQN8VVPWPUI9EQ8LR0YB0HQ2Q9WAYABIHHH3IEXF169MSGD2M37WFI4LWY2D136V7PY8ZPY9344JTW8L3DMS96IVCAPB3AXYCJK0BTHE01X2F0WNBGTQGLR0IPEOF637L8W9HLODUD6A0OLWSNFSJ6VCU6Y04FRUZQEYNT6IOS79DP6WXUTM6ZYMCEIUVAQCVIG1P94VABXJKW07YBW5HT5FT1B7E0SZ5EK3EX6HZUSD0PDUS49X5H8SDNSYANP98MHKNN7AGZ0546U3E

In [28]:
x

```python
class ActionFileData:
  id: str = acfe2005d7004143960a75164b366c2d

```

In [27]:
# x = BlobFile.upload_from_path("somefile.txt", client)

Uploading


Uploading file part 1/1: 100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4144.57it/s]


In [30]:
# %debug

In [13]:
# # syft absolute
# from syft.store.blob_storage import BlobStorageClientConfig
# from syft.store.blob_storage import BlobStorageConfig
# from syft.store.blob_storage.seaweedfs import SeaweedFSClient
# from syft.store.blob_storage.seaweedfs import SeaweedFSClientConfig

# blob_config = BlobStorageConfig(
#     client_type=SeaweedFSClient,
#     client_config=SeaweedFSClientConfig(
#         host="http://0.0.0.0",
#         port="8333",
#         access_key="admin",
#         secret_key="admin",
#         bucket_name="test_bucket",
#         region="us-east-1",
#         # mount_port=4001
#     ),
# )
# node.python_node.init_blob_storage(blob_config)

In [14]:
# syft absolute
from syft.types.blob_storage import BlobFile

In [8]:
x = BlobFile.upload_from_path("somefile.txt", client)

In [15]:
x = BlobFile.upload_from_path("somefile4gb.txt", client)

ConnectionError: Failed to fetch metadata. Response returned with code 500

In [9]:
x.read()

b'abcdef'

In [ ]:
"somefile.txt"

In [35]:
client.api.services.blob_storage.mount_azure(
    account_name="helmprojectstorage",
    container_name="helm",
    account_key=os.environ["HELM_STORAGE_ACCOUNT_KEY"],
    bucket_name="helmazurebucket",
)

SyftSuccess: Mounting Azure Successful!

In [42]:
files = client.api.services.blob_storage.get_files_from_bucket("helmazurebucket")

In [43]:
files

[syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile]

In [44]:
file = [f for f in files if f.file_name == "test.json"][0]

In [45]:
file

```python
class BlobFile:
  id: str = cb2ef738082c49418ed70eb05a193770
  file_name: str = "test.json"

```

In [46]:
file.read()

b'{\n"abc": "def"\n}'